In [128]:
"""
In this script pre-processed data from selected sets are combined, additional population data is added,
and two merged csv files one with DOSE population data and one with ISIMIP population data is created
as a base file from which to produce the paper figures. 

Important instructions: 
This script is meant to be run three times, one for each of the three differing conversions that we will use here. 

1, 2, 3a - restrained by values that overlap with DOSE
1, 2, 3b - converted to log scale 
1, 2, 3c - using isimip population data 

This will create three different csv files from which the figures can be produced.
    - merged_dose_pop.csv
    - merged_isimip_pop.csv
    - merged_loglog.csv

Authors: Josh Arky, Leon Liessem, Matthias Zarama Giedion, Luuk Staal, Brielle Wells
"""

import os
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

""" 1. Load and merge data from different sets """

# Define paths
data_path               =   '../Data/'
deflator_path           =    data_path +'deflator/'
pickle_path             =    data_path +'pickle/'
population_path        =    data_path +'population/'

# Define files
dose_v2   = 'DOSE_V2.10.csv'
pkl_files = ['C2022_data.pkl', 'K2025_data.pkl',
             'Z2024_data.pkl','WS2022_data.pkl','S2024_data.pkl']
isimip_pop = 'isimip.csv'

isimip_data = pd.read_csv(population_path+isimip_pop)
isimip_data['year'] = isimip_data['year'].astype(int)
data = pd.read_csv(data_path+dose_v2)
data['year'] = data['year'].astype(int)

# Load each .pkl file and merge it with the data dataframe
for pkl_file in pkl_files:
    file_path = os.path.join(pickle_path, pkl_file)
    with open(file_path, 'rb') as file:
        pkl_data = pickle.load(file)
        pkl_data['year'] = pkl_data['year'].astype(int)
        # Drop unnecessary columns before merging
        pkl_data = pkl_data.drop(columns=['GID_0'], errors='ignore') #drop GID_0 to not duplicat columns 
        data = pd.merge(data, pkl_data, on=['GID_1', 'year'], how='outer')


# Regenerate GID_0 from the first three characters of GID_1
data['GID_0'] = data['GID_1'].str[:3]

#print(data.columns)

data = data.drop(
    columns=data.filter(regex='^(ag_|man_|serv_)').columns.tolist()
    + ['cpi_2015', 'deflator_2015', 'T_a', 'P_a'], errors='ignore')

#print(data.columns)

data['isimip_pop'] = pd.merge(data, isimip_data, on=['GID_1', 'year'], how='left')['var']
data.columns = data.columns.str.replace('GRP', 'grp', regex=False)
data.columns = data.columns.str.replace('suleiman', 'S2024_grp', regex=False)

print(data.columns)

#replace isimpop 0 with 0.1 to avoid division by zero in future calculations #8591 occurences of 0 in isimip - these are largely small island regions
data['isimip_pop'] = data['isimip_pop'].replace(0, 0.1)

data = data.dropna(subset=['year'])

Index(['country', 'region', 'GID_0', 'GID_1', 'year', 'grp_lcu', 'pop',
       'grp_pc_lcu', 'grp_pc_usd', 'grp_pc_lcu_2015', 'grp_pc_usd_2015',
       'grp_pc_lcu2015_usd', 'fx', 'PPP', 'StructChange', 'version', 'C2022',
       'C2022_grp_ppp_2015', 'C2022_grp_usd_2015', 'C2022_grp_lcu2015_usd',
       'C2022_grp_lcu_2015', 'C2022_pc', 'C2022_grp_pc_ppp_2015',
       'C2022_grp_pc_lcu2015_ppp', 'C2022_grp_pc_usd_2015', 'K2025',
       'K2025_pc', 'K2025_pc_ppp_2015', 'K2025_pc_lcu2015_ppp',
       'K2025_pc_usd_2015', 'K2025_pc_lcu2015_usd', 'Z2024_pc',
       'Z2024_grp_pc_lcu', 'Z2024_grp_pc_lcu_2015', 'Z2024_grp_pc_lcu2015_usd',
       'Z2024_grp_pc_ppp_2015', 'Z2024_grp_pc_lcu2015_ppp', 'Z2024',
       'Z2024_grp_lcu', 'Z2024_grp_lcu_2015', 'Z2024_grp_lcu2015_usd',
       'Z2024_grp_ppp_2015', 'Z2024_grp_lcu2015_ppp', 'WS2022',
       'WS2022_ppp_2015', 'WS2022_lcu2015_ppp', 'WS2022_usd_2015',
       'WS2022_lcu2015_usd', 'WS2022_pc', 'WS2022_pc_ppp_2015',
       'WS2022_pc_lcu20

In [129]:
""" 2. Prepare GRP conversion and deflation for comparison """

# Load World Bank GDP deflator data
deflators = pd.read_excel(deflator_path+'2022_03_30_WorldBank_gdp_deflator.xlsx', sheet_name='Data',
                            index_col=None, usecols='B,E:BM', skiprows=3).set_index('Country Code')
deflators = deflators.dropna(axis=0, how='all')

# Create two dataframes from the deflators, with 2015 and 2005 as ref years
deflators_2017 = deflators.copy()
column2017 = deflators_2017.columns.get_loc(2017)
for i in range(len(deflators_2017)):
    deflators_2017.iloc[i, :] = (deflators_2017.iloc[i, :] / deflators_2017.iloc[i, column2017]) * 100

deflators_2005 = deflators.copy()
column2005 = deflators_2005.columns.get_loc(2005)
for i in range(len(deflators_2005)):
    deflators_2005.iloc[i, :] = (deflators_2005.iloc[i, :] / deflators_2005.iloc[i, column2005]) * 100

# Make separate dataframes for the US and local GDP deflators, both stacked
deflators_2017_us = pd.DataFrame(deflators_2017.loc[deflators_2017.index=='USA'].stack()
                                 ).rename(columns={0:'deflator_2017_us'}
                                          ).reset_index().set_index('level_1')
deflators_2017 = pd.DataFrame(deflators_2017.stack()).rename(columns={0:'deflator_2017'})

deflators_2005_us = pd.DataFrame(deflators_2005.loc[deflators_2005.index=='USA'].stack()
                                 ).rename(columns={0:'deflator_2005_us'}
                                          ).reset_index().set_index('level_1')
deflators_2005 = pd.DataFrame(deflators_2005.stack()).rename(columns={0:'deflator_2005'})

# Add the deflators to the 'data' dataframe
data['deflator_2017'] = np.nan
data['deflator_2017_us'] = np.nan
data['deflator_2005'] = np.nan
data['deflator_2005_us'] = np.nan

print(data.columns)

data = data.set_index(['GID_0','year'])
data.update(deflators_2017)
data.update(deflators_2005)
data = data.reset_index('GID_0')
data.update(deflators_2017_us)
data.update(deflators_2005_us)

# Load PPP conversion factors for both relevant years ...
# ... and add both sets of factors for each country to 'data'
ppp_data = pd.read_excel(deflator_path+'ppp_data_all_countries.xlsx')
ppp_data_2005 = ppp_data.loc[ppp_data.year==2005]
ppp_data_2017 = ppp_data.loc[ppp_data.year==2017]
d2005 = dict(zip(ppp_data_2005.iso_3, ppp_data_2005.PPP))
d2017 = dict(zip(ppp_data_2017.iso_3, ppp_data_2017.PPP))
data['ppp_2005'] = data['GID_0'].apply(lambda x: d2005.get(x))
data['ppp_2017'] = data['GID_0'].apply(lambda x: d2017.get(x))
data.loc[data.GID_0 == 'USA', 'ppp_2005'] = 1
data.loc[data.GID_0 == 'USA', 'ppp_2017'] = 1
data['ppp_2005'] = pd.to_numeric(data['ppp_2005'], errors='coerce')
data['ppp_2017'] = pd.to_numeric(data['ppp_2017'], errors='coerce')

#Create PPP column and print head
data['PPP'] = pd.to_numeric(data.PPP, errors='coerce')
data.sample(60)

# Load MER conversion factors for 2015 and add them for each country to 'data'
fx_data = pd.read_excel(deflator_path+'fx_data_all_countries.xlsx')
fx_data = fx_data.loc[fx_data.year==2005]
d = dict(zip(fx_data.iso_3, fx_data.fx))
data['fx_2005'] = data['GID_0'].apply(lambda x: d.get(x))
data.loc[data.GID_0=='USA','fx_2005'] = len(data.loc[data.GID_0=='USA']) * [1]

print("Current index:", data.index)
print(data.columns)

Index(['country', 'region', 'GID_0', 'GID_1', 'year', 'grp_lcu', 'pop',
       'grp_pc_lcu', 'grp_pc_usd', 'grp_pc_lcu_2015', 'grp_pc_usd_2015',
       'grp_pc_lcu2015_usd', 'fx', 'PPP', 'StructChange', 'version', 'C2022',
       'C2022_grp_ppp_2015', 'C2022_grp_usd_2015', 'C2022_grp_lcu2015_usd',
       'C2022_grp_lcu_2015', 'C2022_pc', 'C2022_grp_pc_ppp_2015',
       'C2022_grp_pc_lcu2015_ppp', 'C2022_grp_pc_usd_2015', 'K2025',
       'K2025_pc', 'K2025_pc_ppp_2015', 'K2025_pc_lcu2015_ppp',
       'K2025_pc_usd_2015', 'K2025_pc_lcu2015_usd', 'Z2024_pc',
       'Z2024_grp_pc_lcu', 'Z2024_grp_pc_lcu_2015', 'Z2024_grp_pc_lcu2015_usd',
       'Z2024_grp_pc_ppp_2015', 'Z2024_grp_pc_lcu2015_ppp', 'Z2024',
       'Z2024_grp_lcu', 'Z2024_grp_lcu_2015', 'Z2024_grp_lcu2015_usd',
       'Z2024_grp_ppp_2015', 'Z2024_grp_lcu2015_ppp', 'WS2022',
       'WS2022_ppp_2015', 'WS2022_lcu2015_ppp', 'WS2022_usd_2015',
       'WS2022_lcu2015_usd', 'WS2022_pc', 'WS2022_pc_ppp_2015',
       'WS2022_pc_lcu20

In [130]:
""" 3. Perform GRP pc conversion and deflation calculations (restricted by population data in DOSE)"""

# DOSE (current lcu)
data['grp_pc_ppp_2017'] = data['grp_pc_lcu'] * 100 / data['deflator_2017'] / data['ppp_2017']
data['grp_pc_ppp_2005'] = data['grp_pc_lcu'] * 100 / data['deflator_2005'] / data['ppp_2005']
# grp_pc_usd already exists in DOSE
data['grp_pc_lcu_2017'] = data['grp_pc_lcu'] * 100 / data['deflator_2017']


# Z2024 (current usd)
data['Z2024_grp_pc_ppp'] = data['Z2024_pc'] * data.fx / data.PPP
data['Z2024_grp_pc_lcu_2017'] = data['Z2024_pc'] * data.fx * 100 / data['deflator_2017']

# K2025 (2017 int USD) to
data['K2025_grp_pc_lcu_2017'] = data['K2025_pc'] * data['ppp_2017']
data['K2025_grp_pc_ppp'] = data['K2025_grp_pc_lcu_2017'] / 100 * data['deflator_2017'] / data.PPP

# C2022 (2017 int USD)
data['C2022_grp_pc_ppp'] = data['C2022_pc'] / 100 * data['deflator_2017_us']


# W&S in 2005 ppp
data['WS2022_grp_pc_ppp'] = data['WS2022_pc'] / 100 * data['deflator_2005_us']


#NOW the same only for total values:


# DOSE (current lcu)
data['grp_ppp_2017'] = data['grp_lcu'] * 100 / data['deflator_2017'] / data['ppp_2017']
data['grp_ppp_2005'] = data['grp_lcu'] * 100 / data['deflator_2005'] / data['ppp_2005']
data['grp_usd'] = data['grp_lcu'] / data.fx
data['grp_lcu_2017'] = data['grp_lcu'] * 100 / data['deflator_2017']

# Z2024 (current usd)
data['Z2024_grp_ppp'] = data['Z2024'] * data.fx / data.PPP
data['Z2024_grp_lcu_2017'] = data['Z2024'] * data.fx * 100 / data['deflator_2017']

# K2025 (2017 int USD) to
data['K2025_grp_lcu_2017'] = data['K2025'] * data['ppp_2017']
data['K2025_grp_ppp'] = data['K2025_grp_lcu_2017'] / 100 * data['deflator_2017'] / data.PPP


# C2022 (2017 int USD)
data['C2022_grp_ppp'] = data['C2022'] / 100 * data['deflator_2017_us']


# W&S in 2005 ppp
data['WS2022_grp_ppp'] = data['WS2022'] / 100 * data['deflator_2005_us']


# export
output_file = f"{data_path}/merged_dose_pop.csv"
data.to_csv(output_file, index=True)
print(f"Data exported successfully to {output_file}")
# print("Current index:", data.index)
# print(data.columns)


Data exported successfully to ../Data//merged_dose_pop.csv


In [124]:
'''3b. - log log conversions  '''




# DOSE (current lcu)
data['grp_pc_ppp_2017'] = data['grp_pc_lcu'] * 100 / data['deflator_2017'] / data['ppp_2017']
data['log_grp_pc_ppp_2017'] = np.log(data['grp_pc_ppp_2017'])
data['grp_pc_ppp_2005'] = data['grp_pc_lcu'] * 100 / data['deflator_2005'] / data['ppp_2005']
data['log_grp_pc_ppp_2005'] = np.log(data['grp_pc_ppp_2005'])
# grp_pc_usd already exists in DOSE
data['log_grp_pc_usd'] = np.log(data['grp_pc_usd'])
data['grp_pc_lcu_2017'] = data['grp_pc_lcu'] * 100 / data['deflator_2017']
data['log_grp_pc_lcu_2017'] = np.log(data['grp_pc_lcu_2017'])


# Z2024 (current usd)
data['log_Z2024_pc'] = np.log(data['Z2024_pc'])
data['Z2024_grp_pc_ppp'] = data['Z2024_pc'] * data.fx / data.PPP
data['log_Z2024_grp_pc_ppp'] = np.log(data['Z2024_grp_pc_ppp'])
data['Z2024_grp_pc_lcu_2017'] = data['Z2024_pc'] * data.fx * 100 / data['deflator_2017']
data['log_Z2024_grp_pc_lcu_2017'] = np.log(data['Z2024_grp_pc_lcu_2017'])




# K2025 (2017 int USD) to
data['log_K2025_pc'] = np.log(data['K2025_pc'])
data['K2025_grp_pc_lcu_2017'] = data['K2025_pc'] * data['ppp_2017']
data['log_K2025_grp_pc_lcu_2017'] = np.log(data['K2025_grp_pc_lcu_2017'])
data['K2025_grp_pc_ppp'] = data['K2025_grp_pc_lcu_2017'] / 100 * data['deflator_2017'] / data.PPP
data['log_K2025_grp_pc_ppp'] = np.log(data['K2025_grp_pc_ppp'])




# C2022 (2017 int USD)
data['log_C2022_pc'] = np.log(data['C2022_pc'])
data['C2022_grp_pc_ppp'] = data['C2022_pc'] / 100 * data['deflator_2017_us']
data['log_C2022_grp_pc_ppp'] = np.log(data['C2022_grp_pc_ppp'])




# W&S in 2005 ppp
data['log_WS2022_pc'] = np.log(data['WS2022_pc'])
data['WS2022_grp_pc_ppp'] = data['WS2022_pc'] / 100 * data['deflator_2005_us']
data['log_WS2022_grp_pc_ppp'] = np.log(data['WS2022_grp_pc_ppp'])


#NOW the same only for total values:




# DOSE (current lcu)
data['grp_ppp_2017'] = data['grp_lcu'] * 100 / data['deflator_2017'] / data['ppp_2017']
data['log_grp_ppp_2017'] = np.log(data['grp_ppp_2017'])
data['grp_ppp_2005'] = data['grp_lcu'] * 100 / data['deflator_2005'] / data['ppp_2005']
data['log_grp_ppp_2005'] = np.log(data['grp_ppp_2005'])
data['grp_usd'] = data['grp_lcu'] / data.fx
data['log_grp_usd'] = np.log(data['grp_usd'])
data['grp_lcu_2017'] = data['grp_lcu'] * 100 / data['deflator_2017']
data['log_grp_lcu_2017'] = np.log(data['grp_lcu_2017'])




# Z2024 (current usd)
data['log_Z2024'] = np.log(data['Z2024'])
data['Z2024_grp_ppp'] = data['Z2024'] * data.fx / data.PPP
data['log_Z2024_grp_ppp'] = np.log(data['Z2024_grp_ppp'])
data['Z2024_grp_lcu_2017'] = data['Z2024'] * data.fx * 100 / data['deflator_2017']
data['log_Z2024_grp_lcu_2017'] = np.log(data['Z2024_grp_lcu_2017'])


# K2025 (2017 int USD) to
data['log_K2025'] = np.log(data['K2025'])
data['K2025_grp_lcu_2017'] = data['K2025'] * data['ppp_2017']
data['log_K2025_grp_lcu_2017'] = np.log(data['K2025_grp_lcu_2017'])
data['K2025_grp_ppp'] = data['K2025_grp_lcu_2017'] / 100 * data['deflator_2017'] / data.PPP
data['log_K2025_grp_ppp'] = np.log(data['K2025_grp_ppp'])




# C2022 (2017 int USD)
data['log_C2022'] = np.log(data['C2022'])
data['C2022_grp_ppp'] = data['C2022'] / 100 * data['deflator_2017_us']
data['log_C2022_grp_ppp'] = np.log(data['C2022_grp_ppp'])


# W&S in 2005 ppp
data['log_WS2022'] = np.log(data['WS2022'])
data['WS2022_grp_ppp'] = data['WS2022'] / 100 * data['deflator_2005_us']
data['log_WS2022_grp_ppp'] = np.log(data['WS2022_grp_ppp'])

# export
output_file = f"{data_path}/merged_loglog.csv"
data.to_csv(output_file, index=True)
print(f"Data exported successfully to {output_file}")




/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/envs/myenv/lib/python3.11/site-packages/panda

Data exported successfully to ../Data//merged_loglog.csv


In [118]:
"""
3b (ISIMIP alternative): Perform GRP pc conversion and deflation calculations using ISIMIP pop data
2025-04-11: results look really crazy when using ISIMIP data... RMSPD through the roof. Need to double-check what's going on
"""

# DOSE (current lcu)
data['grp_pc_ppp'] = data['grp_lcu'] / data.PPP / data['isimip_pop']
data['grp_pc_usd'] = data['grp_lcu'] / data.fx / data['isimip_pop']
data['grp_pc_lcu_2017'] = data['grp_lcu'] * 100 / data['deflator_2017'] / data['isimip_pop']
data['grp_pc_ppp_2005'] = data['grp_pc_ppp'] * 100 / data['deflator_2005_us']
data['grp_pc_ppp_2017'] = data['grp_pc_ppp'] * 100 / data['deflator_2017_us']

# Z2024 (current usd) to current PPP and 2017 LCU
data['Z2024_grp_pc_ppp'] = data['Z2024_pc'] * data.fx / data.PPP
data['Z2024_grp_pc_lcu_2017'] = data['Z2024_pc'] * data.fx * 100 / data['deflator_2017']

# K2025 (2017 int USD) to 2017 LCU and current PPP
data['K2025_grp_pc_lcu_2017'] = data['K2025_pc'] * data['ppp_2017']  ##we need to verify this to original K2025 unit
data['K2025_grp_pc_ppp'] = data['K2025_grp_pc_lcu_2017'] / 100 * data['deflator_2017'] / data.PPP

# C2022 (2017 int USD) to current PPP
data['C2022_grp_pc_ppp'] = data['C2022'] / 100 * data['deflator_2017_us'] / data['isimip_pop']

# W&S in 2005 ppp to current PPP
data['WS2022_grp_pc_ppp'] = data['WS2022'] / data['isimip_pop'] / 100 * data['deflator_2005_us']

print(data.columns)

# export
output_file = f"{data_path}/merged_isimip_pop.csv"
data.to_csv(output_file, index=True)
print(f"Data exported successfully to {output_file}")

print("Current index:", data.index)

Index(['GID_0', 'country', 'region', 'GID_1', 'grp_lcu', 'pop', 'grp_pc_lcu',
       'grp_pc_usd', 'grp_pc_lcu_2015', 'grp_pc_usd_2015',
       'grp_pc_lcu2015_usd', 'fx', 'PPP', 'StructChange', 'version', 'C2022',
       'C2022_grp_ppp_2015', 'C2022_grp_usd_2015', 'C2022_grp_lcu2015_usd',
       'C2022_grp_lcu_2015', 'C2022_pc', 'C2022_grp_pc_ppp_2015',
       'C2022_grp_pc_lcu2015_ppp', 'C2022_grp_pc_usd_2015', 'K2025',
       'K2025_pc', 'K2025_pc_ppp_2015', 'K2025_pc_lcu2015_ppp',
       'K2025_pc_usd_2015', 'K2025_pc_lcu2015_usd', 'Z2024_pc',
       'Z2024_grp_pc_lcu', 'Z2024_grp_pc_lcu_2015', 'Z2024_grp_pc_lcu2015_usd',
       'Z2024_grp_pc_ppp_2015', 'Z2024_grp_pc_lcu2015_ppp', 'Z2024',
       'Z2024_grp_lcu', 'Z2024_grp_lcu_2015', 'Z2024_grp_lcu2015_usd',
       'Z2024_grp_ppp_2015', 'Z2024_grp_lcu2015_ppp', 'WS2022',
       'WS2022_ppp_2015', 'WS2022_lcu2015_ppp', 'WS2022_usd_2015',
       'WS2022_lcu2015_usd', 'WS2022_pc', 'WS2022_pc_ppp_2015',
       'WS2022_pc_lcu2015_ppp',